<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

## Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [1]:
!pip install gensim --upgrade

     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import gensim

gensim.__version__

'4.2.0'

In [3]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
#wv['data']

# a = np.array([[1, 3, 5]])
# a
# b = [[1, 3, 5]]
# b
# c = np.array([1, 3, 5])
# c

### 1. Word2Vec과 코사인 유사도

word2vec을 이용해 구한 'data'와 'science'임베딩 값의 코사인 유사도를 구해보겠습니다. sklearn의 cosine_similarity를 이용하겠습니다.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([wv['data']], [wv['science']])

array([[0.1575913]], dtype=float32)

문항 1) 위에서 구한 코사인 유사도를 소수점 3째자리까지 입력하세요


0.157

### 2. 텍스트 분류

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [ ]:
from google.colab import files

file = files.upload()

In [ ]:
import pandas as pd
## call data set
df = pd.read_csv('spam.csv', encoding = 'latin-1')
df.head()

In [ ]:
## remove unnecessary columns
df = df.iloc[:, [0,1]]
df.head()

In [ ]:
print(df['v1'].value_counts())

In [ ]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encoded = le.fit_transform(df['v1'])

df['v1'] = encoded

df.head()

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [ ]:
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Split data set
X_train, X_test, y_train, y_test = train_test_split(df['v2'], df['v1'], test_size=0.15, train_size=0.85, random_state=42)

## keras 의 tokenizer 에 학습
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

# 단업 집합 vocab 의 크기 입력
vocab_size = len(tokenizer.word_index) + 1   # 패딩을 고려하여 tokenizer의 단어 수에서 +1
print(vocab_size)

## pad_sequence 통해 패딩 처리
X_encoded = tokenizer.texts_to_sequences(X_train)

max_len = 150
X_train = pad_sequences(X_encoded, maxlen=max_len)


## word2vec 의 임베딩 가중치 행렬 생성
embedding_matrix = np.zeros((vocab_size, 300))

print(np.shape(embedding_matrix))


In [ ]:
## keras 모듈을 불러온 후, 학습
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], 
                    input_length=max_len, trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)


In [ ]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)

X_test = pad_sequences(X_test_encoded, maxlen=max_len)

model.evaluate(X_test, y_test)

### 3)Word2Vec에서의 OOV 문제

```
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
```
Lecture Note에 있는 위의 코드를 변형하여, OOV 개수를 확인해주세요.
- tokenizer는 위에서 활용한 tokenizer를 그대로 사용하겠습니다.
- Tip : dictionary를 활용하거나, Counter를 활용해보세요.

In [ ]:
### 이곳에서 과제를 수행해 주세요 ###

oov = []

def get_vector(word):
    if word in wv:
        return wv[word]
    else:
        # return oov.append(word)   <- 가능은 하지만 def 안에 oov가 있어서 abnormal
        return None

for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
    # else:
    #     oov.append(temp)   <- 이러면 oov에 none 만 인쇄됨! 
    else:
        oov.append(word)    

print(len(oov))
print(oov)